# The ***GroupBy*** Pattern

We have seen the ***GroupBy*** operator in ***Pandas***, but  this is actually a more general ***design pattern*** that can be utilized in many data analyics frameworks and data access interfaces, e.g. in ***SQL***. 

## GroupBy: general Pattern
<img SRC="IMG/groupby.jpg">

#### GroupBy in SQL:
```
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
ORDER BY COUNT(CustomerID) DESC;
```

#### GroupBy in MongoDB
```
db.BusinessProcess.aggregate({
    "$group": {
        _id: {
            status: "$status",
            type: "$type"
        },
        count: {
            $sum: 1
        }
    }
   })
```


In [1]:
#setup example
import numpy as np
import pandas as pd
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.786204,-0.752417
1,a,two,-0.658818,0.508703
2,b,one,2.033441,0.643848
3,b,two,-0.995456,-0.472565
4,a,one,0.612878,-0.672272


In [2]:
#group by key1 
grouped = df.groupby(df['key1'])
grouped #this is now a more complex group object

In [3]:
#and generates a table per group
for name, group in grouped:
    print ("name:", name, "\n",group)

name: a 
   key1 key2     data1     data2
0    a  one -0.786204 -0.752417
1    a  two -0.658818  0.508703
4    a  one  0.612878 -0.672272
name: b 
   key1 key2     data1     data2
2    b  one  2.033441  0.643848
3    b  two -0.995456 -0.472565


In [4]:
#access group table
grouped.get_group('b')

,key1,key2,data1,data2
2,b,one,2.033441,0.643848
3,b,two,-0.995456,-0.472565


In [5]:
#get numper of entries (rows) per group
grouped.size()

key1
a    3
b    2
dtype: int64

In [6]:
#get number of group entries by columns
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


#### Think of  grouped DataFrames as 3d objects:

In [7]:
#accessing the "3d" group tables
grouped['data2'].get_group('a')

0   -0.752417
1    0.508703
4   -0.672272
Name: data2, dtype: float64

In [8]:
grouped.get_group('a')['data2']

0   -0.752417
1    0.508703
4   -0.672272
Name: data2, dtype: float64

### Group by external keys

In [9]:
#define external key years as numpy array
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([years]).mean()


2005   -0.813493
2006    1.323160
Name: data1, dtype: float64

### Group by functions

In [10]:
#sort by column and retun top n
def top(df, n=5, column='data1'):
    return df.sort_values(by=column)[-n:]

df.groupby(df['key1']).apply(top,n=5)

key1 key2     data1     data2
key1                                
a    0    a  one -0.786204 -0.752417
     1    a  two -0.658818  0.508703
     4    a  one  0.612878 -0.672272
b    3    b  two -0.995456 -0.472565
     2    b  one  2.033441  0.643848

### Group-wise aggregation (apply)
<img SRC="IMG/groupby.jpg">

#### Typical build in aggregation functions:
* sum
* mean
* max / min
* quantile
* ...

In [11]:
#aggregate over the groups
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


In [12]:
grouped.sum()

,data1,data2
key1,,
a,-0.832144,-0.915986
b,1.037985,0.171283


#### Custom Aggregation Functions

In [13]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.399083,1.261119
b,3.028897,1.116413


#### Multiple aggregations

In [14]:
#just call a list of function
grouped.agg([peak_to_peak, 'mean', 'median'])

data1                            data2                    
     peak_to_peak      mean    median peak_to_peak      mean    median
key1                                                                  
a        1.399083 -0.277381 -0.658818     1.261119 -0.305329 -0.672272
b        3.028897  0.518992  0.518992     1.116413  0.085642  0.085642

#### Suppressing the Group Keys 

In [15]:
df.groupby(df['key1']).apply(top,n=2)

key1 key2     data1     data2
key1                                
a    1    a  two -0.658818  0.508703
     4    a  one  0.612878 -0.672272
b    3    b  two -0.995456 -0.472565
     2    b  one  2.033441  0.643848

In [16]:
df.groupby(df['key1'], group_keys=False).apply(top,n=2)

,key1,key2,data1,data2
1,a,two,-0.658818,0.508703
4,a,one,0.612878,-0.672272
3,b,two,-0.995456,-0.472565
2,b,one,2.033441,0.643848


## More  Exercises in the Lab session... 